In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Work/data_cleaning') ## change the current directory based on the local system

In [ ]:
!pip install xlsxwriter ## install xlswriter if not installed

     |████████████████████████████████| 153kB 4.3MB/s 


In [ ]:
import pandas as pd
import numpy as np
import time
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from nltk.corpus import wordnet as wn
from nltk.corpus import words
import nltk.tag, nltk.data
import spacy
import os
import sys

from collections import Counter
from math import sqrt
import datetime as dt
import logging
import re
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import enum
import openpyxl
lemm = WordNetLemmatizer()
path='en'
nlp = spacy.load(path, disable=['parser', 'ner'])
stop = stopwords.words('english')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Functions for data cleaning

In [ ]:
def basicTextProcessing(row):
    utterance = str(row["utterance"]).lower()
    intent = str(row["intent"])
    intent = intent.replace(' ', '')
    utterance = utterance.translate(str.maketrans('', '', string.punctuation))
    utterance = " ".join(utterance.split())
    row["utterance"] = utterance
    row["intent"] = intent
    return row


def removeStopWords(row):
    utterance = row["utterance"]
    tokens = nltk.word_tokenize(utterance)
    sent = ""
    for w in tokens:
        if re.search('[a-zA-Z]', str(w)) != None and not w.lower() in stop:
            sent = sent + " " + w.lower()
    utterance = sent
    row["utterance"] = utterance
    return row

def get_tokens(utterance):
    
    tokens = nltk.word_tokenize(utterance)
    
    return tokens
    
def get_wordnet_pos(treebank_tag):
    
    if treebank_tag.startswith('J'):
        return wn.ADJ
    elif treebank_tag.startswith('V'):
        return wn.VERB
    elif treebank_tag.startswith('N'):
        return wn.NOUN
    elif treebank_tag.startswith('R'):
        return wn.ADV
    else:
        return wn.NOUN


def lemmatizeTokens(row):
    utterance = row["utterance"]
    tokens = nltk.word_tokenize(utterance)
    sent = ""
    pos = nltk.pos_tag(tokens)
    for num, word in enumerate(pos, start=0):
        pos_value = word[1]
        word_value = word[0]
        pos_word = get_wordnet_pos(pos_value)
        lemm_token = lemm.lemmatize(word_value, pos_word)
        if lemm_token.endswith("ing"):
            word_ing.add(lemm_token)
        sent = sent + " " + lemm_token
    sent = sent.strip()
    #sent = " ".join(sent.split())
    utterance = sent
    row["utterance"] = utterance
    return row


def lemmatizeTokens_spacy(raw_data_lemm, ignoreTokenList):
    #raw_data_lemm = raw_data.copy()
    df_temp = pd.DataFrame()
    for value, row in raw_data_lemm.iterrows():
        
        try:
            #biGrams = biGram_DF
            #triGrams = triGram_DF
            utterance = row["utterance"]
            #utterance = "iservice ??"
    
            n = 3
            sixgrams = nltk.ngrams(utterance.split(), n)
            ignoreWords = set()
            for grams in sixgrams:
                #print(grams)
                bGram_1 = grams[0] + " " + grams[1]
                bGram_2 = grams[1] + " " + grams[2]
                
                triGram_text = grams[0] + " " + grams[1] + " " + grams[2]
                if triGram_text in ignoreTokenList:
                    ignoreWords.add(grams[0])
                    ignoreWords.add(grams[1])
                    ignoreWords.add(grams[2])
                elif bGram_1 in ignoreTokenList:
                    ignoreWords.add(grams[0])
                    ignoreWords.add(grams[1])
                elif bGram_2 in ignoreTokenList:
                    ignoreWords.add(grams[1])
                    ignoreWords.add(grams[2])
    
        
            doc = nlp(utterance)
            sent = ""
            
            for token in doc:
                #print(token)
                #token = doc[1]
                
                
                if token not in ignoreWords and str(token) not in ignoreTokenList and re.search('[a-zA-Z]', str(token)) != None: #and str(Configurations.GeneralConfig.i_Language) == str(Configurations.Language.English)
                    #print(token)
                    lemm_token = token.lemma_
                else:
                    lemm_token = token
                #if str(lemm_token).endswith("ing"):
                    #word_ing.add(token)
                if str(lemm_token) == "-PRON-":
                    lemm_token = token
                sent = sent + " " + str(lemm_token)
            sent = sent.strip()
            #sent = " ".join(sent.split())
            utterance = sent
            row["utterance"] = utterance
            df_temp = df_temp.append(row)
        except Exception as e:
            print(e)
            print(utterance)
            break
        
    return df_temp

### Clenaning of Train data

In [ ]:
df_train=pd.read_excel('train_irisdata.xlsx',sheet_name='Train',engine='openpyxl')
ignoreTokenList=[loc.lower() for loc in pd.read_csv('ngrams.csv')["ngram"].dropna()]

In [ ]:
df_train['utterance'] = df_train['utterance'].str.lower()
df_train['intent'] = df_train['intent'].str.lower()

In [ ]:
df_train = df_train.apply(basicTextProcessing, axis = 1)
lemmatizeTokens_spacy(df_train, ignoreTokenList)
df_train = df_train.apply(removeStopWords, axis = 1)
df_train['utterance']=df_train['utterance'].str.split().str[-30:].str.join(' ')

### Cleaning of Test data

In [ ]:
df_test=pd.read_excel('test_irisdata.xlsx',engine='openpyxl')
df_test['utterance'] = df_test['utterance'].str.lower()
df_test['intent'] = df_test['intent'].str.lower()

In [ ]:
df_test = df_test.apply(basicTextProcessing, axis = 1)
lemmatizeTokens_spacy(df_test, ignoreTokenList)
df_test = df_test.apply(removeStopWords, axis = 1)
df_test['utterance']=df_test['utterance'].str.split().str[-30:].str.join(' ')

generator raised StopIteration
avalog


In [ ]:
df_test

,utterance,intent
0,add new user access functional would,access_management.add_new_users_to_access_func...
1,add new user access fid,access_management.add_new_users_to_access_func...
2,apply access funcional account uams,access_management.apply_access_functional_id_i...
3,apply uams coordinator,access_management.apply_for_uams_coordinator
4,approver leave uams,access_management.how_to_perform_delegation_in...
...,...,...
3487,zoom installation,applications_systems.zoom
3488,zoom software,applications_systems.zoom
3489,zoom usage,applications_systems.zoom
3490,zoomed browser,computer.zoomed_browser


### Labelling

In [ ]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()
le.fit(df_train['intent'])

df_train['intent1']=le.transform(df_train['intent'])
mapping=df_train[['intent','intent1']].drop_duplicates().rename(columns={'intent1':'mapping'})

df_test=pd.merge(df_test,mapping,on='intent',how='left').rename(columns={'mapping':'intent1'})

df_test['intent1']=df_test['intent1'].fillna(0).astype(int)

In [ ]:
df_test

,utterance,intent,intent1
0,add new user access functional would,access_management.add_new_users_to_access_func...,4
1,add new user access fid,access_management.add_new_users_to_access_func...,4
2,apply access funcional account uams,access_management.apply_access_functional_id_i...,8
3,apply uams coordinator,access_management.apply_for_uams_coordinator,10
4,approver leave uams,access_management.how_to_perform_delegation_in...,23
...,...,...,...
3487,zoom installation,applications_systems.zoom,76
3488,zoom software,applications_systems.zoom,76
3489,zoom usage,applications_systems.zoom,76
3490,zoomed browser,computer.zoomed_browser,217


## Save the cleaned data

In [ ]:
writer=pd.ExcelWriter('test_iris_fullcleaned.xlsx',engine='xlsxwriter')
df_test.to_excel(writer,sheet_name='Test',index=None)
mapping.to_excel(writer,sheet_name='Mapping',index=None)
writer.save()

In [ ]:
writer=pd.ExcelWriter('train_iris_fullcleaned.xlsx',engine='xlsxwriter')
df_train.to_excel(writer,sheet_name='Train',index=None)
writer.save()